In [1]:
from image_composer_test import compose_ad_frame

In [2]:
import os
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from autogen import Agent

def compose_ad_frame(assets_folder, concept_data):
    """
    Composes an advertisement frame using assets and concept data.
    
    Parameters:
    - assets_folder (str): Path to the folder containing image assets.
    - concept_data (dict): Data describing the concept and assets for the frame.
    
    Returns:
    - PIL.Image.Image: Composed advertisement frame.
    """
    # Initialize a blank canvas for the ad frame
    frame = Image.new('RGB', (600, 400), color='white')
    draw = ImageDraw.Draw(frame)
    
    # Load and paste landing and endframe images
    landing_image = Image.open(os.path.join(assets_folder, '0a59be2e7dd53d6de11a10ce3649c081/landing_1.png'))
    endframe_image = Image.open(os.path.join(assets_folder, '0a59be2e7dd53d6de11a10ce3649c081/end-1.jpg'))
    frame.paste(landing_image, (0, 0))
    frame.paste(endframe_image, (400, 0))
    
    # Example of manually setting text and color
    font = ImageFont.load_default()
    draw.text((50, 50), concept_data['text'], font=font, fill='white')
    primary_color = concept_data['primary_color']
    draw.rectangle([(0, 0), (200, 100)], fill=primary_color)
    
    return frame

In [3]:
import json
import os
import random
import time
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import matplotlib.pyplot as plt
import requests
from PIL import Image
from termcolor import colored

import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.llava_agent import LLaVAAgent, llava_call
import replicate

LLAVA_MODE = "remote"  # Either "local" or "remote"
assert LLAVA_MODE in ["local", "remote"]

In [4]:
import os

if LLAVA_MODE == "remote":

    llava_config_list = [
        {
            "model": "whatever, will be ignored for remote",  # The model name doesn't matter here right now.
            "api_key": os.environ["REPLICATE_API_TOKEN"],
            "base_url": "yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591",
        }
    ]

In [5]:
assistant = autogen.AssistantAgent(
    name="assistant",
    code_execution_config=False,
    system_message=
    """

    """,

    llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": {"config_list": llava_config_list, "temperature": 0.5, "max_new_tokens": 1000},
        "functions": [
             {
                        "name": "ask_worker",
                        "description": "ask worker to:  1. generate questions for requirement gathering, 2. retrieve relevant laws based on the topic sent as message",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "message": {
                                    "type": "string",
                                    "description": "question to ask worker. Make sure the question include enough context, such as the required task. The worker does not know the conversation between you and the user, unless you share the conversation with the worker. Ensure the original question of the user is also passed as context such that the worker is generating intermediate steps necessary to address it. It is VERY IMPORTANT that you provide a clear and autoritative feedback to the worker to stay focused on answering the original question. If the worker is wandering off from the mission, terminate!"
                                },
                                "message_type": {
                                    "type": "string",
                                    "description": "Type of work you want it to achieve. Options are: 'REQUIREMENT GATHERING','SIMPLE QUESTION ANSWERING'"
                                },
                                "retrival":{
                                    "type": "string",
                                    "description": "Becomes 'TRUE' or 'FALSE' which would indicate to perform retrieval of relevant articles from the law to achieve the task"
                                },
                            },
                            "required": ["message","message_type","retrival"]
                        }
                        }
        ],
    },
)

AttributeError: 'str' object has no attribute 'copy'

In [ ]:
import os
from PIL import Image, ImageDraw, ImageFont

def compose_ad_frame(assets_folder, concept_data):
    """
    Composes an advertisement frame using assets and concept data.
    
    Parameters:
    - assets_folder (str): Path to the folder containing image assets.
    - concept_data (dict): Data describing the concept and assets for the frame.
      Expected keys:
        - 'text': Text to be displayed on the frame.
        - 'primary_color': Primary color for elements on the frame.
        - 'text_position': Position tuple (x, y) for the text.
        - 'rectangle_size': Size tuple (width, height) for the colored rectangle.
    
    Returns:
    - PIL.Image.Image: Composed advertisement frame.
    """
    # Initialize a blank canvas for the ad frame
    frame = Image.new('RGB', (600, 400), color='white')
    draw = ImageDraw.Draw(frame)
    
    # Load and paste landing and endframe images
    landing_image = Image.open(os.path.join(assets_folder, '0a59be2e7dd53d6de11a10ce3649c081/landing_1.png'))
    endframe_image = Image.open(os.path.join(assets_folder, '0a59be2e7dd53d6de11a10ce3649c081/end-1.jpg'))
    frame.paste(landing_image, (0, 0))
    frame.paste(endframe_image, (400, 0))
    
    # Extract concept data
    text = concept_data.get('text', '')
    #primary_color = concept_data.get('primary_color', '#000000')
    text_position = concept_data.get('text_position', (50, 50))
    rectangle_size = concept_data.get('rectangle_size', (200, 100))
    
    # Draw text on the frame
    font = ImageFont.load_default()
    draw.text(text_position, text, font=font, fill='black')
    
    # Draw colored rectangle
    draw.rectangle([(0, 0), rectangle_size])#, fill=primary_color)
    
    return frame

In [6]:
assets_folder = 'data'
concept_data = {
    'text': 'Sample advertisement text',
    #'primary_color': '#000000',
    'text_position': (500, 50),
    'rectangle_size': (200, 100)
}

composed_frame = compose_ad_frame(assets_folder, concept_data)
composed_frame.show()

KeyError: 'primary_color'

In [7]:
assistant = autogen.AssistantAgent(
    name="ad_composer",
    code_execution_config=False,
    system_message=
    """
    You are an Advertisement Composer Assistant with advanced capabilities to compose advertisement frames dynamically based on user requirements.
    
    Your primary objective is to adjust the parameters of the advertisement frame based on feedback received from the CriticAgent or other sources.
    
    Available Tasks:
    - COMPOSE_ADVERTISEMENT_FRAME: Compose an advertisement frame using given concept data.
    
    Your tasks:
    1. Receive feedback on the current advertisement frame composition.
    2. Adjust the 'text', 'primary_color', 'text_position', and 'rectangle_size' parameters in the concept data based on feedback.
    3. Ensure that the adjustments reflect the desired changes for improving the advertisement frame.
    4. Return the updated advertisement frame.
    
    Guidelines:
    - Use the 'COMPOSE_ADVERTISEMENT_FRAME' task to delegate the composition process.
    - Dynamically adjust the concept data parameters based on feedback before composing the advertisement frame.
    - Implement changes based on specific feedback to achieve better composition results.
    
    Communication:
    - Communicate with other agents or users to gather necessary feedback.
    - Ensure the advertisement frame meets the required criteria specified by the feedback received.
    - If additional adjustments are needed, repeat the process until satisfactory results are achieved.
    
    If you need to ask a question or you have finished the task, add TERMINATE at the end.
    """,
    llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": llava_config_list,  # Add configuration list if needed
        "functions": [
            {
                "name": "COMPOSE_ADVERTISEMENT_FRAME",
                "description": "Compose an advertisement frame using given concept data.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "assets_folder": {
                            "type": "string",
                            "description": "Path to the folder containing image assets."
                        },
                        "concept_data": {
                            "type": "object",
                            "description": "Data describing the concept and assets for the advertisement frame.",
                            "properties": {
                                "text": {
                                    "type": "string",
                                    "description": "Text to be displayed on the frame."
                                },
                                "primary_color": {
                                    "type": "string",
                                    "description": "Primary color for elements on the frame."
                                },
                                "text_position": {
                                    "type": "array",
                                    "items": {
                                        "type": "integer"
                                    },
                                    "description": "Position tuple (x, y) for the text."
                                },
                                "rectangle_size": {
                                    "type": "array",
                                    "items": {
                                        "type": "integer"
                                    },
                                    "description": "Size tuple (width, height) for the colored rectangle."
                                }
                            },
                            "required": ["text", "primary_color", "text_position", "rectangle_size"]
                        }
                    },
                    "required": ["assets_folder", "concept_data"]
                }
            }
        ]
    }
)

In [8]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    #is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    # code_execution_config={"work_dir": "planning"},
    function_map={"COMPOSE_ADVERTISEMENT_FRAME": compose_ad_frame },
)

In [9]:
def chat_with_agent(message):
  # combined_message, lang = translate_to_dest(message,"en")
  user_proxy.initiate_chat(
      assistant,
      #clear_history=False,
      #silent=True,
      message=message
  )
  last_message = [messages for agent, messages in user_proxy.chat_messages.items()][0][-1]["content"]#.replace("TERMINATE" , "")
  # result_message, lang = translate_to_dest(last_message, lang)
  # print(result_message)
  print(last_message)

In [11]:
!pip install ipywidgets


[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [12]:
# !pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize the list to hold messages
messages = []


def handle_input(sender):
    global messages
    user_input = sender.value
    messages.append(f"User: {user_input}")
    chat_with_agent(user_input)
    messages.append("")
    display_messages()

# Function to display messages
def display_messages():

    text_input = widgets.Text(
        value='',
        placeholder='Type your message',
        description='Chat:',
        disabled=False
    )
    text_input.on_submit(handle_input)
    display(text_input)


display_messages()

/tmp/ipykernel_84574/2419385609.py:26: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  text_input.on_submit(handle_input)


Text(value='', description='Chat:', placeholder='Type your message')

user_proxy (to ad_composer):

data

--------------------------------------------------------------------------------


APIConnectionError: Connection error.

In [8]:
import cv2

image1 = cv2.imread('data/0a59be2e7dd53d6de11a10ce3649c081/cta.png') 
image2 = cv2.imread('data/0a59be2e7dd53d6de11a10ce3649c081/landing_1.png')

cv2.imshow('Image', image1)

error: OpenCV(4.10.0) /io/opencv/modules/highgui/src/window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
